## Fast generation of Kronecker graphs in Julia
These codes are fairly heavily optimized to show how a careful implementation of the ideas in our paper can give rise to particularly fast generation of Kronecker graphs. The basic idea is the same, the codes have just been optimized in a number of ways.

The most different routine is the one to count multiset permutations. This new code does not build the counter representation of a multiset. Instead, what it does is carefully update the number of permutations after each swap. It turns out that this is faster (in Julia) than building the counter representation.

The other change is that we go to some lengths to avoid _division_. When we profiled the code, the division operations were taking a substantial portion. This could still be optimized more, we believe.

In [1]:
## Regions

# Increment to the next region of a Kronecker graph
@inline function _next_region!{T <: Integer}(cur::AbstractArray{T,1}, m::Integer)
  lastk = length(cur)
  for k=length(cur):-1:1
    lastk = k              # save the last state
    cur[k] += 1            # increment the last element
    if cur[k] == m+1       # if there is spill
      if k > 1             # if there is an array
        continue # recur on prefix
      else
        cur[k] = 0
      end
    end
    break
  end
  for k=lastk+1:length(cur)
    cur[k] = cur[k-1]
  end
  cur
end
cur = [1,1,1,1]
@show _next_region!(cur, 3)
@show _next_region!(cur, 3)
@show _next_region!(cur, 3)
@show _next_region!(cur, 3)
@show _next_region!(cur, 3)
@time _next_region!(cur, 3)

##
cur = [1,1,1]
@show _next_region!(cur, 4)
while cur[1] != 0
  @show _next_region!(cur, 4)
end


##
function _first_region(a::AbstractArray,k::Integer)
  return ones(UInt32,k)
end

function _all_regions(a::AbstractArray,k::Integer)
  cur = _first_region(a,k)
  rval = Array(typeof(cur),0)
  while cur[1] != 0
    push!(rval, copy(cur))
    _next_region!(cur, 4)
  end
  return rval
end

@show R =_all_regions([0.99,0.5,0.5,0.2],3)
@show typeof(R)
@time _all_regions([0.99,0.5,0.5,0.2],3)
@time _all_regions([0.99,0.5,0.5,0.2],4)
@time _all_regions([0.99,0.5,0.5,0.2],5)



_next_region!(cur, 3) = [1, 1, 1, 2]
_next_region!(cur, 3) = [1, 1, 1, 3]
_next_region!(cur, 3) = [1, 1, 2, 2]
_next_region!(cur, 3) = [1, 1, 2, 3]
_next_region!(cur, 3) = [1, 1, 3, 3]
  0.000005 seconds (84 allocations: 6.436 KiB)
_next_region!(cur, 4) = [1, 1, 2]
_next_region!(cur, 4) = [1, 1, 3]
_next_region!(cur, 4) = [1, 1, 4]
_next_region!(cur, 4) = [1, 2, 2]
_next_region!(cur, 4) = [1, 2, 3]
_next_region!(cur, 4) = [1, 2, 4]
_next_region!(cur, 4) = [1, 3, 3]
_next_region!(cur, 4) = [1, 3, 4]
_next_region!(cur, 4) = [1, 4, 4]
_next_region!(cur, 4) = [2, 2, 2]
_next_region!(cur, 4) = [2, 2, 3]
_next_region!(cur, 4) = [2, 2, 4]
_next_region!(cur, 4) = [2, 3, 3]
_next_region!(cur, 4) = [2, 3, 4]
_next_region!(cur, 4) = [2, 4, 4]
_next_region!(cur, 4) = [3, 3, 3]
_next_region!(cur, 4) = [3, 3, 4]
_next_region!(cur, 4) = [3, 4, 4]
_next_region!(cur, 4) = [4, 4, 4]
_next_region!(cur, 4) = [0, 0, 0]


Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{Array{UInt32,1}}, ::Int64) at ./deprecated.jl:57
 [3] _all_regions(::Array{Float64,1}, ::Int64) at ./In[1]:46
 [4] include_string(::String, ::String) at ./loading.jl:515
 [5] include_string(::Module, ::String, ::String) at /home/dgleich/.julia/v0.6/Compat/src/Compat.jl:407
 [6] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/dgleich/.julia/v0.6/IJulia/src/execute_request.jl:154
 [7] eventloop(::ZMQ.Socket) at /home/dgleich/.julia/v0.6/IJulia/src/eventloop.jl:8
 [8] (::IJulia.##14#17)() at ./task.jl:335
while loading In[1], in expression starting on line 243


R = _all_regions([0.99, 0.5, 0.5, 0.2], 3) = Array{UInt32,1}[UInt32[0x00000001, 0x00000001, 0x00000001], UInt32[0x00000001, 0x00000001, 0x00000002], UInt32[0x00000001, 0x00000001, 0x00000003], UInt32[0x00000001, 0x00000001, 0x00000004], UInt32[0x00000001, 0x00000002, 0x00000002], UInt32[0x00000001, 0x00000002, 0x00000003], UInt32[0x00000001, 0x00000002, 0x00000004], UInt32[0x00000001, 0x00000003, 0x00000003], UInt32[0x00000001, 0x00000003, 0x00000004], UInt32[0x00000001, 0x00000004, 0x00000004], UInt32[0x00000002, 0x00000002, 0x00000002], UInt32[0x00000002, 0x00000002, 0x00000003], UInt32[0x00000002, 0x00000002, 0x00000004], UInt32[0x00000002, 0x00000003, 0x00000003], UInt32[0x00000002, 0x00000003, 0x00000004], UInt32[0x00000002, 0x00000004, 0x00000004], UInt32[0x00000003, 0x00000003, 0x00000003], UInt32[0x00000003, 0x00000003, 0x00000004], UInt32[0x00000003, 0x00000004, 0x00000004], UInt32[0x00000004, 0x00000004, 0x00000004]]
typeof(R) = Array{Array{UInt32,1},1}
  0.003157 seconds (2.

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{Array{UInt32,1}}, ::Int64) at ./deprecated.jl:57
 [3] _all_regions(::Array{Float64,1}, ::Int64) at ./In[1]:46
 [4] include_string(::String, ::String) at ./loading.jl:515
 [5] include_string(::Module, ::String, ::String) at /home/dgleich/.julia/v0.6/Compat/src/Compat.jl:407
 [6] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/dgleich/.julia/v0.6/IJulia/src/execute_request.jl:154
 [7] eventloop(::ZMQ.Socket) at /home/dgleich/.julia/v0.6/IJulia/src/eventloop.jl:8
 [8] (::IJulia.##14#17)() at ./task.jl:335
while loading In[1], in expression starting on line 237


56-element Array{Array{UInt32,1},1}:
 UInt32[0x00000001, 0x00000001, 0x00000001, 0x00000001, 0x00000001]
 UInt32[0x00000001, 0x00000001, 0x00000001, 0x00000001, 0x00000002]
 UInt32[0x00000001, 0x00000001, 0x00000001, 0x00000001, 0x00000003]
 UInt32[0x00000001, 0x00000001, 0x00000001, 0x00000001, 0x00000004]
 UInt32[0x00000001, 0x00000001, 0x00000001, 0x00000002, 0x00000002]
 UInt32[0x00000001, 0x00000001, 0x00000001, 0x00000002, 0x00000003]
 UInt32[0x00000001, 0x00000001, 0x00000001, 0x00000002, 0x00000004]
 UInt32[0x00000001, 0x00000001, 0x00000001, 0x00000003, 0x00000003]
 UInt32[0x00000001, 0x00000001, 0x00000001, 0x00000003, 0x00000004]
 UInt32[0x00000001, 0x00000001, 0x00000001, 0x00000004, 0x00000004]
 UInt32[0x00000001, 0x00000001, 0x00000002, 0x00000002, 0x00000002]
 UInt32[0x00000001, 0x00000001, 0x00000002, 0x00000002, 0x00000003]
 UInt32[0x00000001, 0x00000001, 0x00000002, 0x00000002, 0x00000004]
 ⋮                                                                 
 UInt32[0x0

In [2]:


## Unranking

# unrank([1,2,2,4], 1) returns [1,2,2,4]
# unrank([1,2,2,4], 2) returns [1,2,4,2]
# unrank([0,1,1,3], 3) returns [1,4,2,2]

# This assumes ms is sorted

function num_multiset_permutations{T<:Integer}(ms::AbstractArray{T,1}, start_pos::Integer = 1)
  nperm = 1
  count = 1
  countfac = 1
  @inbounds for i in start_pos+1:length(ms)
    nperm *= (i-start_pos+1)
    if ms[i] == ms[i-1]
      count += 1
      countfac *= count
    else
      nperm = div(nperm,countfac)
      # new element
      count = 1
      countfac = 1
    end
  end
  nperm = div(nperm,countfac)
end

# Find the next distinct character after a[k]
# Returns the index where it occurs or length(a)+1
@inline function _next_distinct_character(a, k::Integer, m::Integer)
  # check for easy out
  if k>=m
    return m+1
  else
    ak = a[k]
    if ak != a[k+1] # many cases
      return k+1
    elseif ak == a[m] # some cases
      return m+1
    else
      # we have to search, so there are probably many dups
      # since we assume sorted, we can bisect
      lo = 1
      hi = m-k # we need to search [k+lo:k+hi]
      while hi - lo > 2
        mid = (hi + lo) >> 1
        if ak == a[k+mid]
          lo = mid
        else
          hi = mid
        end
      end
      ak != a[k+lo+1] && return k+lo+1
      return k+lo+2
    end
  end
end

@show _next_distinct_character([5,6,6,6,8],2,4)
@show _next_distinct_character([5,6,6,6,6,8],1,6)
@show _next_distinct_character([5,6,6,6,6,8],6,6)-6

# This version doesn't use recursion, and unranks the sorted set ms in place
# It also gets the multiplicity of each element to update nperm without
# recomputing it from scratch, it also can take in the total number
# of permutations and just update as we go.
##
# This version doesn't use recursion, and unranks the sorted set ms in place
# It also gets the multiplicity of each element to update nperm without
# recomputing it from scratch, it also can take in the total number
# of permutations and just update as we go.
# This version tries to avoid integer division by storing nperm as a float
@inline function unrank5!{T <: Integer}(ms::AbstractArray{T,1}, n::Integer, nperm, m)
  multcur = 0
  imultcur = 1.0
  multnext = 0
  npermf = float(nperm)
  @inbounds for k=1:m # for each location in the output
    if n == 1
      break
    end

    if multcur == 0 # update
      multcur = _next_distinct_character(ms, k, m) - k # multiplicity of the current element
      imultcur = 1.0/multcur
    end

    # update nperm for removing multcur
    npermf *= multcur
    npermf /= (m-k+1) # shrink the length
    nperm = round(typeof(nperm),npermf)

    #=
    if nperm != num_multiset_permutations(ms, k+1)
      @show 0, ms, k, num_multiset_permutations(ms, k+1), nperm
    end
    @assert nperm == num_multiset_permutations(ms, k+1)
    =#

    if n <= nperm # this means we start with ms[k]
      multcur -= 1
      if multcur != 0
        imultcur = 1.0/multcur
      end
      continue
    else # we need to search
      n -= nperm

      if k+multcur <= m
        if ms[k] == ms[k+multcur]
          #@show ms, k, _next_distinct_character(ms,k,m), multcur
          @assert ms[k] != ms[k+multcur]
        end
      end

      pos = k+multcur
      while pos <= m
        # we are going to swap ms[k] and ms[pos]
        # but to update, we need to know the multiplicity of ms[pos]

        multnext = _next_distinct_character(ms,pos,m) - pos

        # so the element ms[pos] has multiplicity multnext
        # and element ms[k] has multiplicity multcur-1

        ms[pos], ms[k] = ms[k], ms[pos] # swap leading positions
        npermf *= multnext
        npermf *= imultcur
        nperm = round(typeof(nperm),npermf)

        #=
        if nperm != num_multiset_permutations(ms, k+1)
          @show 1, ms, k, num_multiset_permutations(ms, k+1), nperm
        end
        @assert nperm == num_multiset_permutations(ms, k+1)
        =#

        if n <= nperm
          break
        else
          n -= nperm
          pos += multnext
          multcur = multnext
          imultcur = 1.0/multcur
        end
      end

      multcur = 0 # reset the search
    end
  end
  return ms
end
unrank5!(ms, n) = unrank5!(ms, n, num_multiset_permutations(ms), length(ms))

for i=1:12
  @show unrank5!([0,1,2,2], i)
end
@show unrank5!([0,1,2,2], 13)

##
@show unrank5!([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2], 8)
@show unrank5!([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2], 8, num_multiset_permutations([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2]), 18)
@show unrank5!([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2], 8, 18, 18)


_next_distinct_character([5, 6, 6, 6, 8], 2, 4) = 5
_next_distinct_character([5, 6, 6, 6, 6, 8], 1, 6) = 2
_next_distinct_character([5, 6, 6, 6, 6, 8], 6, 6) - 6 = 1
unrank5!([0, 1, 2, 2], i) = [0, 1, 2, 2]
unrank5!([0, 1, 2, 2], i) = [0, 2, 1, 2]
unrank5!([0, 1, 2, 2], i) = [0, 2, 2, 1]
unrank5!([0, 1, 2, 2], i) = [1, 0, 2, 2]
unrank5!([0, 1, 2, 2], i) = [1, 2, 0, 2]
unrank5!([0, 1, 2, 2], i) = [1, 2, 2, 0]
unrank5!([0, 1, 2, 2], i) = [2, 0, 1, 2]
unrank5!([0, 1, 2, 2], i) = [2, 0, 2, 1]
unrank5!([0, 1, 2, 2], i) = [2, 1, 0, 2]
unrank5!([0, 1, 2, 2], i) = [2, 1, 2, 0]
unrank5!([0, 1, 2, 2], i) = [2, 2, 0, 1]
unrank5!([0, 1, 2, 2], i) = [2, 2, 1, 0]
unrank5!([0, 1, 2, 2], 13) = [2, 0, 1, 2]
unrank5!([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2], 8) = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1]
unrank5!([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2], 8, num_multiset_permutations([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]), 18) = [1, 1, 1, 1, 1, 1,

18-element Array{Int64,1}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 2
 1
 1
 1
 1
 1
 1
 1

In [4]:


##
@inline function mydivmod(val,n)
  if n==2
    d2 = val >> 1
    d1 = val - d2*n
  elseif n==4
    d2 = val >> 2
    d1 = val - d2*n
  else
    d1 = mod(val,n)
    d2 = div(val,n)
  end
  return d2, d1
end
@show mydivmod(5,2), divrem(5,2)

##
@inline function _morton_decode{T <: Integer}(mind::AbstractArray{T,1}, n::Integer)
  row = 0
  rowbase = 1
  col = 0
  colbase = 1
  invn = 1.0/n
  for val in mind
    val -= 1
    #d2,d1 = fldmod(val,n) # this is faster tham divrem
    #d1 = mod(val,n)
    #d2 = div(val,n)
    d2,d1 = mydivmod(val,n)
    row += rowbase*d1
    rowbase *= n
    col += colbase*d2
    colbase *= n
  end
  return (row+1,col+1)
end
@show _morton_decode([4,4,4,4],2)
@time _morton_decode([4,4,4,4],2)

(mydivmod(5, 2), divrem(5, 2)) = ((2, 1), (2, 1))
_morton_decode([4, 4, 4, 4], 2) = (16, 16)
  0.000006 seconds (6 allocations: 304 bytes)


(16, 16)

In [5]:
##
_randgeo(p) = ceil(Int,-randexp() / log1p(-p)) # jsut a copy of Distributions.jl
function _generate_region!{Ta,Tb <: Integer}(
    ei::Array{Ta}, ej::Array{Ta},
    r::AbstractArray{Tb,1},
    e::AbstractArray{Tb,1},
    p, small_n::Integer)

    N = num_multiset_permutations(r) # total area
    m = length(r)
    i = 0

    # ugh, do we need this? should make these tolerances.
    if N*p < 1.0e-12 || p < 1.0e-15
      return
    end

    gap = _randgeo(p)

    while i+gap <= N
      i += gap
      copy!(e, r)
      #@show e, i, N, m
      mult_edge = unrank5!(e, i, N, m) # generate the ith permutation
      src, dst = _morton_decode(mult_edge, small_n)
      push!(ei, src)
      push!(ej, dst)
      gap = _randgeo(p)
    end
end
##


function fast_kron_edges(K,k)
  n = size(K,1)
  v = vec(K) # vectorized
  ei = zeros(Int,0)
  ej = similar(ei)
  expected_edges = ceil(Int, 1.05*(sum(v)^k - trace(K)^k))
  sizehint!(ei, expected_edges)
  sizehint!(ej, expected_edges)

  nregions = 0
  region = _first_region(v, k)
  edge_in_region = similar(region) # allocated once to avoid lots of allocs
  while region[1] != 0
    nregions += 1
    # compute the probability
    p = 1.0
    for j in region
      p *= v[j]
    end
    _generate_region!(ei, ej, region, edge_in_region, p, n)
    _next_region!(region, length(v))
  end
  return ei, ej
end

function kron_graph(K,k)
  ei,ej = fast_kron_edges(K,k)
  return sparse(ei,ej,1,2^k,2^k)
end

srand(1)
@assert maximum(kron_graph([0.99 0.5; 0.5 0.2],6))==1
@assert maximum(kron_graph([0.99 0.5; 0.5 0.2],15))==1


@time fast_kron_edges([0.99 0.5; 0.5 0.2], 2)
#@time fast_kron_edges([0.99 0.5; 0.5 0.2], 14)

#Profile.clear_malloc_data()

@time fast_kron_edges([0.99 0.5; 0.5 0.2], 18)

##
@time fast_kron_edges([0.99 0.5; 0.5 0.02], 18)

##
K = [0.99 0.5; 0.5 0.2]
Profile.clear()
@profile fast_kron_edges(K, 18);
Profile.print()


  0.000024 seconds (17 allocations: 1024 bytes)
  0.584169 seconds (15 allocations: 21.514 MiB, 0.53% gc time)
  0.107989 seconds (15 allocations: 4.595 MiB)
636 ./task.jl:335; (::IJulia.##14#17)()
 636 ...Julia/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
  636 ...rc/execute_request.jl:154; execute_request(::ZMQ.Socket, ::...
   636 .../Compat/src/Compat.jl:407; include_string(::Module, ::Strin...
    636 ./loading.jl:515; include_string(::String, ::String)
     636 ./<missing>:?; anonymous
      636 ./profile.jl:23; macro expansion
       2   ./In[5]:44; fast_kron_edges(::Array{Float6...
       634 ./In[5]:53; fast_kron_edges(::Array{Float6...
        12  ./In[5]:22; _generate_region!(::Array{Int...
         4 ./array.jl:0; copy!(::Array{UInt32,1}, ::In...
         7 ./array.jl:134; copy!(::Array{UInt32,1}, ::In...
          5 ./array.jl:120; unsafe_copy!(::Array{UInt32,1...
           5 ./array.jl:113; unsafe_copy!
        492 ./In[5]:24; _generate_region!(::Array{Int...
         2  